In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import Boson
import Proca as Proca
from scipy.integrate import solve_ivp
Boson.general()
Proca.general()

In [ ]:
#Primero un caso sencillo con una estrella de bosones 1D.
LambT = 1  # autointeracción: 1 (repulsiva), 0, -1 (atractiva)
nodos = 1 #Numero de nodos, solo estamos en 1D por lo tanto basta con uno
p0x, p1x = 1, 0  # valores inciliales de sigma y de su primer derivada
u1x = 0  #Primera derivada del potencial

Iniciales = [p0x, p1x, u1x ] 
rmax = 25  #radio máximo

#Semilla para el potencial
Uxint = [0.1, 3]
datos = []
U0x,rTmax, _  = Boson.Bosshot(Iniciales, Uxint, rmax, LambT=LambT,
                                                n=nodos, lim=1e-8, info=False, klim=1500, outval=10, delta=0.2)

# plot the result
U0 = [p0x, p1x, U0x, u1x]
Boson.plotPerfBos(U0, rTmax, LambT)

In [ ]:
#Ya podemos activar los demás nodos
LambT = 0  # autointeracción: 1 (repulsive), 0 (without selinteraction), -1 (attractive)

nx, ny, nz = 2, 1, 1  # node numbers: nx -> nodes of \sigma_x, ny  -> nodes of \sigma_y, ...
nodos = [nx, ny, nz]  # list of node info


p0x, p1x = 1, 0  # The central amplitude of \sigma_x (p0x) and its first derivative value (p1x), the latter is zero
p0y, p1y = 1, 0  # The same in y-axes
p0z, p1z = 1, 0      # The same in z-axes
u1x, u1y, u1z = 0, 0, 0  # The first derivatives of the x, y, and z effective potential components are all zero.

Ini0 = [p0x, p1x, p0y, p1y, p0z, p1z, u1x, u1y, u1z]  # boundary conditions list
rmax = 25  # iteration radius

# Range of values used as seed for the effective potential
Uxint = [0.1, 2]
Uyint = [0.1, 2]
Uzint = [0.1, 2]

datos = []
U0x, U0y, U0z, rTmax, _, _, _ = Proca.Shoot_Proca1(Ini0, Uxint, Uyint, Uzint, rmax, LambT=LambT,
                                                nodos=nodos, lim=1e-8, info=False, klim=1500, outval=10, delta=0.2)

# plot the result
U0 = [p0x, p1x, p0y, p1y, p0z, p1z, U0x, u1x, U0y, u1y, U0z, u1z]
Proca.plotPerf(U0, rTmax, LambT)


In [ ]:
#Ahora podemos refinar la solución 
rext = 6  #Cuanto queremos incrementear el radio 
rmin, rmax = 0, rTmax + rext
limit = [rmin, rmax]
p1x, p1y, p1z, u1x, u1y, u1z = 0, 0, 0, 0, 0, 0  #Las condiciones de frontera
uk = [U0x, U0y, U0z]  #nuestras semillas de la solución no refinada

# solving
V0X = [p0x, p1x, p0y, p1y, p0z, p1z, None, u1x, None, u1y, None, u1z]
V0Duk = np.zeros(12*3)
V0Duk[6] = 1
V0Duk[20] = 1
V0Duk[-2] = 1
V0 = np.concatenate((V0X, V0Duk))
print(V0)

In [ ]:
#Ahora podemos refinar la solución 
rext = 6  #Cuanto queremos incrementear el radio 
rmin, rmax = 0, rTmax + rext
limit = [rmin, rmax]
p1x, p1y, p1z, u1x, u1y, u1z = 0, 0, 0, 0, 0, 0  #Las condiciones de frontera
uk = [U0x, U0y, U0z]  #nuestras semillas de la solución no refinada

# solving
V0X = [p0x, p1x, p0y, p1y, p0z, p1z, None, u1x, None, u1y, None, u1z]
V0Duk = np.zeros(12*3)
#La posición de las derivadas no triviales
V0Duk[6] = 1
V0Duk[20] = 1
V0Duk[-2] = 1
V0 = np.concatenate((V0X, V0Duk))

#Posción de las cks que vamos a ajustar
indck = np.concatenate(([False, False, False, False, False, False, True, False, True, False, True, False], [False]*36))
V0[indck] = uk

BCind = [0, 0, 0]

indXc = np.array([False]*48) 
indXc[:5:2] = True  #La posción de las variables para las que queremos resolver.

inddXc = np.array([False]*48) #Las derivadas de las variables y sus posiciones
inddXc[12::12] =  True
inddXc[14::12] =  True
inddXc[16::12] =  True

V0fit = Proca.fitting(Proca.system_refinado, V0, indck, indXc, BCind, inddXc, limit, argf=LambT, tol=1e-10)
U0 = V0fit[:12]
Proca.plotPerf(U0, rmax, LambT, px=True, lim=False)

In [ ]:
#Ahora podemos pasar al calculo de la masa y la energía.
confi = [U0x, U0y, U0z, rmax, [p0x, p0y, p0z], nodos, LambT] #Esta es la configuración general de la estrella
px0, py0, pz0 = confi[4]
rmin, rmax = 0, confi[3]
LambT = confi[-1] 

metodo = 'RK45'; Rtol = 1e-09; Atol = 1e-10
Nptos = 500; rspan = np.linspace(rmin, rmax, Nptos); 

arg = [LambT]
u0x, u0y, u0z = confi[0], confi[1], confi[2]
U0 = [px0, 0, py0, 0, pz0, 0, u0x, 0, u0y, 0, u0z, 0]

sol2 = solve_ivp(Proca.systemProca, [rmin, rmax], U0, t_eval=rspan,
                     args=(arg,), method=metodo, rtol=Rtol, atol=Atol)
rad = sol2.t
px, ux = sol2.y[0], sol2.y[6][0]
py, uy = sol2.y[2], sol2.y[8][0]
pz, uz = sol2.y[4], sol2.y[10][0]

sigtot = px**2 + py**2 + pz**2

_, masaT = Proca.energMul(rad, sigtot, ux)
print(f"La masa es: {masaT}" ) 

In [ ]:
#Los datos de ejemplo para comprobar funcionamiento
datos = [
  [0.9185797787133904, 0.0, 0.0, 20, [1.0, 0.0, 0], [0, 1, 2], 0],
  [0.9096109737806662, 1.4608345383430958, 0.0, 23, [0.99, 0.025, 0], [0, 1, 2], 0],
  [0.9075085207455894, 1.4574355805220829, 0.0, 20.801145652799903, [0.987, 0.05, 0], [0, 1, 2], 0],
  [0.8975962120277898, 1.44147276871182, 0.0, 21.093870287124965, [0.975, 0.075, 0], [0, 1, 2], 0],
  [0.8854098143064014, 1.421840747957605, 0.0, 20, [0.96, 0.1, 0], [0, 1, 2], 0],
  [0.860034412595141, 1.3809870546640286, 0.0, 23, [0.93, 0.125, 0], [0, 1, 2], 0],
  [0.8352695283628023, 1.3410911542449409, 0.0, 20, [0.9, 0.15, 0], [0, 1, 2], 0],
  [0.8021225992772414, 1.2876610797360635, 0.0, 23, [0.86, 0.175, 0], [0, 1, 2], 0],
  [0.7608283331922263, 1.2210832479554825, 0.0, 20, [0.81, 0.2, 0], [0, 1, 2], 0],
  [0.702859111596206, 1.1275343620787186, 0.0, 23, [0.74, 0.225, 0], [0, 1, 2], 0],
  [0.6205277161841714, 0.9945492936294549, 0.0, 20, [0.64, 0.25, 0], [0, 1, 2], 0],
  [0.5628515737223025, 0.9012007021170526, 0.0, 23, [0.57, 0.26, 0], [0, 1, 2], 0],
  [0.5135960474191803, 0.8213994220698124, 0.0, 27.154850032037178, [0.51, 0.265, 0], [0, 1, 2], 0],
  [0.4727913155779859, 0.7551935214469294, 0.0, 28.011943689000802, [0.46, 0.267, 0], [0, 1, 2], 0],
  [0.415531125359361, 0.6621824517247614, 0.0, 33, [0.39, 0.265, 0], [0, 1, 2], 0],
  [0.3593021759210128, 0.5705465319937473, 0.0, 35, [0.32, 0.26, 0], [0, 1, 2], 0],
  [0.2968719876911216, 0.46832403077246226, 0.0, 35, [0.24, 0.25, 0], [0, 1, 2], 0],
  [0.2587505388574897, 0.40568756157789704, 0.0, 35, [0.19, 0.24, 0], [0, 1, 2], 0],
  [0.22656416357286724, 0.352489658409764, 0.0, 35, [0.145, 0.23, 0], [0, 1, 2], 0],
  [0.20031223041291482, 0.3088944767913174, 0.0, 39, [0.105, 0.22, 0], [0, 1, 2], 0],
  [0.17818483844444952, 0.27197960686340544, 0.0, 39, [0.065, 0.21, 0], [0, 1, 2], 0],
  [0.0, 0.24441214593732527, 0.0, 39, [0, 0.202, 0], [0, 1, 2], 0],
  ]

In [ ]:
gamma = 0
alpha = gamma  #Parametro de rescalamiento, puede tomar valores de 0 y 1. Este depende del valor de gamma, el cual nos indica la polarizacion
#La cuestion es que para tener codiciones apropiadas de reescalamiento si gamma=0 alpha=0 y si gamma = 1 alpha = 1
datosSig0EnMas = Proca.MasaEnergia_Datos(datos, gamma)
Proca.Plot_paper(datosSig0EnMas, cmap_colors=['#f0784d', '#2681ab'], colormap_resolution=50)